In [1]:
import cv2
from matplotlib import pyplot as plt
import torch
import open3d as o3d
import numpy as np

In [2]:
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.default_transform

Using cache found in C:\Users\Enrique/.cache\torch\hub\intel-isl_MiDaS_master
Using cache found in C:\Users\Enrique/.cache\torch\hub\facebookresearch_WSL-Images_master


Loading weights:  None


Using cache found in C:\Users\Enrique/.cache\torch\hub\intel-isl_MiDaS_master


In [ ]:
cap= cv2.VideoCapture('C:/Users/Enrique/Desktop/SeniorProject/Videos/IMG_9265.mp4')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('frame'+str(i)+'.png',frame)
    img = cv2.imread('frame'+str(i)+'.png') 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    i+=1
 
cap.release()
cv2.destroyAllWindows()

j=0
while(j < 720):
    img = cv2.imread('frame'+str(j)+'.png') 
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img1 = cv2.imread('frame'+str(j)+'.png') 
    #img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

    input_batch = transform(img)
    
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False,
    ).squeeze()   
        
    j+=30


    output = prediction.cpu().numpy()
    
    fig = plt.imshow(output)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig('frame_depthmap'+str(j)+'.png', bbox_inches='tight', pad_inches = 0)

    fig1 = plt.imshow(img1)
    plt.axis('off')
    fig1.axes.get_xaxis().set_visible(False)
    fig1.axes.get_yaxis().set_visible(False)
    plt.savefig('frame_rgb'+str(j)+'.png', bbox_inches='tight', pad_inches = 0)

#webcam_capture = cv2.VideoCapture(0)
#ret,frame = webcam_capture.read()
#cv2.imwrite('cam_image.png',frame)
#img = cv2.imread("cam_image.png") 
#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


#ret,frame1 = webcam_capture.read()
#cv2.imwrite('cam_image.png',frame1)
#img1 = cv2.imread("cam_image.png") 
#img1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#input_batch = transform(img)

#with torch.no_grad():
#    prediction = midas(input_batch)
#
#    prediction = torch.nn.functional.interpolate(
#        prediction.unsqueeze(1),
#        size=img.shape[:2],
#        mode="bicubic",
#        align_corners=False,
#    ).squeeze()
    
#output = prediction.cpu().numpy()

In [ ]:
#m=30
#while(m <= 720):
color_raw = o3d.io.read_image('C:/Users/Enrique/Desktop/SeniorProject/frame_rgb510.png')
depth_raw = o3d.io.read_image('C/Users/Enrique/Desktop/SeniorProject/frame_depthmap510.png')

color = o3d.geometry.Image(np.array(np.asarray(color_raw)[:, :, :3]).astype('uint8'))
depth = o3d.geometry.Image(np.array(np.asarray(depth_raw)[:, :, :3]).astype('uint8'))

rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color, depth, convert_rgb_to_intensity=False)
#open3d.io.write_image('C:/Users/Enrique/Desktop/SeniorProject/frame_rgbd'+str(m)+'.rgbd')
    #m+=30

    

In [ ]:
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
#o3d.visualization.draw_geometries([pcd])


#pcd.normals = o3d.utility.Vector3dVector(np.zeros(
#    (1, 3))) 
#pcd.estimate_normals()
o3d.visualization.draw_geometries([pcd], point_show_normal=True)